In [5]:
# Pipeline KNN per assegnare il possibile voto agli "indecisi / non voto / scheda bianca/nulla"
import pandas as pd
import numpy as np

from sklearn.compose import ColumnTransformer, make_column_selector as selector
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsClassifier


# 1) Carico il dataset
df=pd.read_csv("/Users/aminginofabbruccibarbagli/Downloads/dataset_clean.csv", sep=';')
df

TARGET = "m_p_int_voto"

# 2) le classi "da predire" (indecisi / non voto / bianca/nulla / preferisco non rispondere)
classi_da_predire = {
    "sono indeciso",
    "non andrei a votare",
    "voterei  scheda bianca / annullerei la scheda",
    "voterei scheda bianca / scheda nulla",
    "preferisco non rispondere",
}

# 3) Split: train = chi ha espresso un partito; to_predict = indecisi/non voto/bianca/nulla
mask_to_predict = df[TARGET].isin(classi_da_predire)
df_train = df[~mask_to_predict].copy()
df_pred  = df[mask_to_predict].copy()

# Rimuovo eventuali target mancanti nel train
df_train = df_train[df_train[TARGET].notna()]

# 4) Feature set: tutte le colonne tranne il target
X_train = df_train.drop(columns=[TARGET])
y_train = df_train[TARGET]
X_pred  = df_pred.drop(columns=[TARGET])

# 5) Costruisco il preprocessing
numeric_processor = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler(with_mean=False))  # with_mean=False per compatibilità con sparse
])

categorical_processor = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=True))
])

preprocess = ColumnTransformer(transformers=[
    ("num", numeric_processor, selector(dtype_include=np.number)),
    ("cat", categorical_processor, selector(dtype_include=object))
], sparse_threshold=1.0)  # forziamo sparse per gestire alta dimensionalità

# 6) Modello KNN (k scelto moderato per stabilità; puoi regolarlo)
knn = KNeighborsClassifier(n_neighbors=25, weights="distance", metric="minkowski", p=2)

model = Pipeline(steps=[
    ("prep", preprocess),
    ("clf", knn)
])

# 7) Addestramento
model.fit(X_train, y_train)

# 8) Predizione su indecisi/non voto/bianca/nulla
pred_labels = model.predict(X_pred)
proba = model.predict_proba(X_pred)
class_labels = model.named_steps["clf"].classes_
max_proba = proba.max(axis=1)

# Top-3 suggerimenti per trasparenza
top3_idx = np.argsort(proba, axis=1)[:, -3:][:, ::-1]
top3_labels = np.take(class_labels, top3_idx)
top3_probs  = np.take_along_axis(proba, top3_idx, axis=1)

top3_str = [
    "; ".join(f"{lab} ({p:.2f})" for lab, p in zip(lbls, probs))
    for lbls, probs in zip(top3_labels, top3_probs)
]

# 9) Creo la tabella risultati
risultati = pd.DataFrame({
    "row_id": X_pred.index,  # indice originale come riferimento
    "risposta_originale": df_pred[TARGET].values,
    "voto_stimato": pred_labels,
    "prob": max_proba,
    "top3_partiti": top3_str
}).sort_values(by="prob", ascending=False).reset_index(drop=True)

# Salvo anche su CSV
out_path = "/Users/aminginofabbruccibarbagli/Downloads/predizioni_indecisi.csv"
risultati.to_csv(out_path, index=False)


# Output sintetico (prime 10 righe) per preview diretta
risultati.head(10)


,row_id,risposta_originale,voto_stimato,prob,top3_partiti
0,2980,sono indeciso,Partito Democratico-PD,0.681263,Partito Democratico-PD (0.68); MoVimento 5 Ste...
1,3848,sono indeciso,Partito Democratico,0.640512,Partito Democratico (0.64); Partito Democratic...
2,660,sono indeciso,Partito Democratico-PD,0.638958,Partito Democratico-PD (0.64); MoVimento 5 Ste...
3,332,sono indeciso,Partito Democratico-PD,0.602255,Partito Democratico-PD (0.60); MoVimento 5 Ste...
4,919,sono indeciso,MoVimento 5 Stelle,0.601065,MoVimento 5 Stelle (0.60); Lega con Salvini (0...
5,1094,sono indeciso,Partito Democratico-PD,0.600943,Partito Democratico-PD (0.60); Partito Democra...
6,1228,sono indeciso,Partito Democratico-PD,0.600738,Partito Democratico-PD (0.60); Partito Democra...
7,2683,sono indeciso,Partito Democratico-PD,0.600051,Partito Democratico-PD (0.60); Partito Democra...
8,699,sono indeciso,Partito Democratico-PD,0.599674,Partito Democratico-PD (0.60); MoVimento 5 Ste...
9,879,sono indeciso,MoVimento 5 Stelle,0.597451,MoVimento 5 Stelle (0.60); Partito Democratico...


In [ ]:
PROVA CON RANDOM FOREST

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# 1) Ricarico dataset (per sicurezza)
df=pd.read_csv("/Users/aminginofabbruccibarbagli/Downloads/dataset_clean.csv", sep=';')

TARGET = "m_p_int_voto"

# Categorie da predire
classi_da_predire = {
    "sono indeciso",
    "non andrei a votare",
    "voterei  scheda bianca / annullerei la scheda",
    "voterei scheda bianca / scheda nulla",
    "preferisco non rispondere",
}

# Split
mask_to_predict = df[TARGET].isin(classi_da_predire)
df_train = df[~mask_to_predict].copy()
df_pred  = df[mask_to_predict].copy()
df_train = df_train[df_train[TARGET].notna()]

X_train = df_train.drop(columns=[TARGET])
y_train = df_train[TARGET]
X_pred  = df_pred.drop(columns=[TARGET])

# Preprocessing (correzione: uso sparse_output invece di sparse)
numeric_processor = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median"))
])

categorical_processor = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=True))
])

preprocess = ColumnTransformer(transformers=[
    ("num", numeric_processor, selector(dtype_include=np.number)),
    ("cat", categorical_processor, selector(dtype_include=object))
], sparse_threshold=1.0)

# Random Forest
rf = RandomForestClassifier(n_estimators=300, random_state=42, n_jobs=-1, class_weight="balanced")

model_rf = Pipeline(steps=[
    ("prep", preprocess),
    ("clf", rf)
])

# Addestramento
model_rf.fit(X_train, y_train)

# Predizioni
pred_labels = model_rf.predict(X_pred)
proba = model_rf.predict_proba(X_pred)
class_labels = model_rf.named_steps["clf"].classes_
max_proba = proba.max(axis=1)

# Top-3
top3_idx = np.argsort(proba, axis=1)[:, -3:][:, ::-1]
top3_labels = np.take(class_labels, top3_idx)
top3_probs  = np.take_along_axis(proba, top3_idx, axis=1)

top3_str = [
    "; ".join(f"{lab} ({p:.2f})" for lab, p in zip(lbls, probs))
    for lbls, probs in zip(top3_labels, top3_probs)
]

# Tabella risultati
risultati_rf = pd.DataFrame({
    "row_id": X_pred.index,
    "risposta_originale": df_pred[TARGET].values,
    "voto_stimato": pred_labels,
    "confidenza": max_proba,
    "top3_partiti": top3_str
}).sort_values(by="confidenza", ascending=False).reset_index(drop=True)

# Salvo su CSV
out_path_rf = ("/Users/aminginofabbruccibarbagli/Downloads/random.csv")
risultati_rf.to_csv(out_path_rf, index=False)



risultati_rf.head(10)


,row_id,risposta_originale,voto_stimato,confidenza,top3_partiti
0,1472,sono indeciso,Partito Democratico-PD,0.470000,Partito Democratico-PD (0.47); MoVimento 5 Ste...
1,788,sono indeciso,Partito Democratico-PD,0.466667,Partito Democratico-PD (0.47); MoVimento 5 Ste...
2,381,sono indeciso,Partito Democratico-PD,0.456667,Partito Democratico-PD (0.46); Partito Democra...
3,3960,sono indeciso,Partito Democratico,0.453333,Partito Democratico (0.45); Partito Democratic...
4,1467,sono indeciso,Partito Democratico-PD,0.450000,Partito Democratico-PD (0.45); MoVimento 5 Ste...
5,487,sono indeciso,Partito Democratico-PD,0.446667,Partito Democratico-PD (0.45); MoVimento 5 Ste...
6,1452,sono indeciso,Partito Democratico-PD,0.433333,Partito Democratico-PD (0.43); MoVimento 5 Ste...
7,1270,preferisco non rispondere,MoVimento 5 Stelle,0.423333,MoVimento 5 Stelle (0.42); Partito Democratico...
8,1504,non andrei a votare,MoVimento 5 Stelle,0.413333,MoVimento 5 Stelle (0.41); Partito Democratico...
9,2683,sono indeciso,Partito Democratico-PD,0.410000,Partito Democratico-PD (0.41); Partito Democra...


In [10]:
from sklearn.model_selection import cross_val_score, StratifiedKFold

# Definiamo train set (solo voti chiari)
df_train_only = df_train.copy()
X = df_train_only.drop(columns=[TARGET])
y = df_train_only[TARGET]

# Preprocessing (compatibile con tua versione di sklearn -> uso sparse=True)
numeric_processor = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler(with_mean=False))
])

categorical_processor = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=True))
])

preprocess = ColumnTransformer(transformers=[
    ("num", numeric_processor, selector(dtype_include=np.number)),
    ("cat", categorical_processor, selector(dtype_include=object))
], sparse_threshold=1.0)

# Modelli
knn = KNeighborsClassifier(n_neighbors=25, weights="distance")
rf = RandomForestClassifier(n_estimators=300, random_state=42, n_jobs=-1, class_weight="balanced")

pipe_knn = Pipeline(steps=[("prep", preprocess), ("clf", knn)])
pipe_rf  = Pipeline(steps=[("prep", preprocess), ("clf", rf)])

# Cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

scores_knn = cross_val_score(pipe_knn, X, y, cv=cv, scoring="accuracy", n_jobs=-1)
scores_rf  = cross_val_score(pipe_rf,  X, y, cv=cv, scoring="accuracy", n_jobs=-1)

# Tabella confronto
confronto = pd.DataFrame({
    "Modello": ["KNN", "Random Forest"],
    "Accuratezza media": [scores_knn.mean(), scores_rf.mean()],
    "Deviazione std": [scores_knn.std(), scores_rf.std()]
})

confronto


/opt/anaconda3/lib/python3.10/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/opt/anaconda3/lib/python3.10/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


,Modello,Accuratezza media,Deviazione std
0,KNN,0.334712,0.002004
1,Random Forest,0.469129,0.016438
